In [ ]:
#!/usr/bin/python3
# coding: utf-8

In [ ]:
from datetime import datetime as dt
import sys
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly

In [ ]:
from reportlab.lib.units import cm, mm
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.platypus import Paragraph, Frame, Image, Table, TableStyle, LongTable
from reportlab.platypus import PageBreak, SimpleDocTemplate, BaseDocTemplate

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')
update_map = False

In [ ]:
with open("csv/longcovid.tmp", "rt") as f:
    prev = int(f.read())
imgname = "longcovid-map.jpg"
tw_body = '新型コロナ 後遺症外来マップ (' + today_str + ')'
df=pd.read_csv("csv/longcovid.csv", encoding='shift-jis', header=0)

In [ ]:
if len(df) > prev:
    update_map = True
    with open("csv/longcovid.tmp", "wt") as f:
        f.write("{}".format(len(df)))

In [ ]:
#update_map = True # TODO: AAWA

In [ ]:
fig = px.scatter_mapbox(df,
    title=tw_body,
    lat="lat", lon="lon", color="可", size="可",
    color_continuous_scale=plotly.colors.sequential.Bluered,
    hover_name="施設名称", 
    hover_data=["オンライン", "紹介状", "条件"],
    labels={'lat':'緯度', 'lon':'経度'},
    size_max=12, zoom=5, height=550)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.update_layout(template='plotly_dark')
if update_map:
    show_and_save_plotly(fig, imgname, js=False, show=True, image=False, html=True)
    appended = len(df) - prev
    tw_body += "\n{}件追加 ".format(appended)
    tw_body += "\nPDF: https://raw.githubusercontent.com/geneasyura/cov19-hm/master/docs/images/longcovid.pdf"
    tw_body += "\n地図: https://geneasyura.github.io/cov19-hm/longcovid.html "
    tw = get_twitter()
    tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)
    print(tw_body)
else:
    print("nothin to tweet about long-covid hospital map.")

In [ ]:
def df_to_html(filename):
    with open(filename, "w", encoding='utf-8') as f:
        f.write('<style>\n')
        f.write('.MapsTblColNo { width:  70px; }\n')
        f.write('.MapsTblColSt { width:  90px; }\n')
        f.write('.MapsTblColNm { width: 280px; }\n')
        f.write('.MapsTblColEn { width:  70px; }\n')
        f.write('.MapsTblColOL { width:  70px; }\n')
        f.write('.MapsTblColRS { width:  70px; }\n')
        f.write('.MapsTblColCd { width: 160px; }\n')
        f.write('.MapsTblColHP { width:  50px; }\n')
        f.write('</style>\n')

        f.write('<table border="0">\n')
        f.write('  <thead style="display: block;" bgcolor="#000000">\n')
        f.write('    <tr style="text-align: left;">\n')
        f.write('      <th class="MapsTblColNo">八地方</th>\n')
        f.write('      <th class="MapsTblColSt">都道府県</th>\n')
        f.write('      <th class="MapsTblColNm">施設名</th>\n')
        f.write('      <th class="MapsTblColEn">診療可</th>\n')
        f.write('      <th class="MapsTblColOL">Web</th>\n')
        f.write('      <th class="MapsTblColRS">紹介状</th>\n')
        f.write('      <th class="MapsTblColCd">条件</th>\n')
        f.write('      <th class="MapsTblColHP">HP</th>\n')
        f.write('    </tr>\n')
        f.write('  </thead>\n')
        f.write('  <tbody style="display: block; overflow-x: hidden; overflow-y: scroll; height: 300px;">\n')
        for i, r in df.iterrows():
            f.write('<tr bgcolor="{}">'.format(["#606060", "#000000"][i % 2]))
            f.write('<td class="MapsTblColNo">{}</td>'.format(r['八地方区分']))
            f.write('<td class="MapsTblColSt">{}</td>'.format(r['都道府県']))
            f.write('<td class="MapsTblColNm">{}</td>'.format(r['施設名称']))
            f.write('<td class="MapsTblColEn">{}</td>'.format(r['可']))
            f.write('<td class="MapsTblColOL">{}</td>'.format(r['オンライン']))
            f.write('<td class="MapsTblColRS">{}</td>'.format(r['紹介状']))
            f.write('<td class="MapsTblColCd">{}</td>'.format(r['条件']))
            f.write('<td class="MapsTblColHP"><a href="{}">HP</a></td>'.format(r['HP']))
            f.write('</tr>\n')
        f.write('</tbody></table>\n')

In [ ]:
df_to_html('docs/_includes/longcovid-table.html')

In [ ]:
font_name = 'HeiseiKakuGo-W5'
page_size = landscape(A4)
pdfFile = SimpleDocTemplate(
    'docs/images/longcovid.pdf',
    #showBoundary=1,
    pagesize=page_size,
    title="新型コロナ 後遺症外来一覧 by 遺伝子組換え阿修羅",
    leftMargin=11*mm, rightMargin=11*mm,
    topMargin=11*mm, bottomMargin=11*mm)
pdfmetrics.registerFont(UnicodeCIDFont(font_name))

In [ ]:
style_dict ={
    "name":"normal",
    "fontName":font_name,
    "fontSize":9
}
body_style = ParagraphStyle(**style_dict)

In [ ]:
story = []
link = '<link href="{}">遺伝子組換え阿修羅 @zwiYWJqourMuEh7</link>'.format("https://twitter.com/zwiYWJqourMuEh7")
story.append(Paragraph(
    "新型コロナ 後遺症外来一覧 by " + link + " " + today_str, body_style))

In [ ]:
pdfFile.author = "遺伝子組換え阿修羅 @zwiYWJqourMuEh7"
pdfFile.subject = "新型コロナ 後遺症外来一覧"
pdfFile.creator = "python3 with reportlab"
pdfFile.keywords = "COVID19, LongCovid, SARS-Cov-2, Japan"

In [ ]:
headers = ['No.', '都道府県', '施設名称', '住所', 'オンライン', '紹介状', '条件', 'HP']
rows = []
rows.append(headers)
for i, r in df.iterrows():
    columns = [
        r['No.'], r['都道府県'], r['施設名称'], r['所在地'],
        r['オンライン'], r['紹介状'], r['条件'],
        Paragraph('<link href="{}">Link</link>'.format(r['HP']), body_style)
    ]
    rows.append(columns)

table = LongTable(rows, colWidths=(10*mm, 15*mm, 75*mm, 55*mm, 20*mm, 15*mm, 60*mm, 15*mm))
table.setStyle(TableStyle([
    ('FONT', (0, 0), (-1, -1), font_name, 9),
    ('BOX', (0, 0), (-1, -1), 0.5, colors.black),
    ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.black),
    ('VALIGN', (0, 0), (-1, -1), 'BOTTOM'),
    ]))

In [ ]:
story.append(table)

In [ ]:
pdfFile.multiBuild(story)